In [2]:
import os
import pandas as pd
import numpy as np
from neo4j import Query, GraphDatabase, RoutingControl, Result # Python database driver 5.13 +

## Database connection

In [3]:
DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "insurance" # Have to be neo4j for neo4j aura (but keep it, good for testing on local dev env)

In [4]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))
driver.verify_connectivity()

In [ ]:
# Ensure database (skip this step on neo4j aura)
records, summary, keys = driver.execute_query(
    "create database {dbname} if not exists".format(dbname = DB_NAME),
    database_="system",
    routing_=RoutingControl.WRITE
)

## Utility functions

In [5]:
## Utility
def split_dataframe(df, chunk_size = 50000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

## Data sources

In [7]:
csv_policy = pd.read_csv('~/import/freMTPLfreq.csv')
csv_policy.head()

,PolicyID,ClaimNb,Exposure,Power,CarAge,DriverAge,Brand,Gas,Region,Density
0,1,0,0.09,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
1,2,0,0.84,g,0,46,Japanese (except Nissan) or Korean,Diesel,Aquitaine,76
2,3,0,0.52,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
3,4,0,0.45,f,2,38,Japanese (except Nissan) or Korean,Regular,Nord-Pas-de-Calais,3003
4,5,0,0.15,g,0,41,Japanese (except Nissan) or Korean,Diesel,Pays-de-la-Loire,60


In [8]:
csv_claim = pd.read_csv('~/import/freMTPLsev.csv')

In [9]:
csv_claim.insert(0, 'ClaimId', range(0, len(csv_claim)))
csv_claim.head()

,ClaimId,PolicyID,ClaimAmount
0,0,63987,1172
1,1,310037,1905
2,2,314463,1150
3,3,318713,1220
4,4,309380,55077


## Schema definition

In [ ]:
schema_statements = [
    'create constraint if not exists for (n:Policy) require (n.id) is node key',
    'create constraint if not exists for (n:Car) require (n.id) is node key',
    'create constraint if not exists for (n:CarModel) require (n.id) is node key',
    'create index if not exists for (n:Car) on (n.brand)',
    'create index if not exists for (n:Car) on (n.age)',
    'create index if not exists for (n:Car) on (n.fuel)',
    'create constraint if not exists for (n:Claim) require (n.id) is node key',
    'create constraint if not exists for (n:Region) require (n.id) is node key',
    'create constraint if not exists for (n:Product) require (n.id) is node key',
    'create constraint if not exists for (n:Coverage) require (n.id) is node key'
]
for statement in schema_statements:
    driver.execute_query(
        statement,
        database_=DB_NAME,
        routing_=RoutingControl.WRITE
    )

# Fetch all constraints
schema_result_df  = driver.execute_query(
    'show constraints',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
schema_result_df.head(100)

## Graph creation

In [ ]:
# Not wrapped in split_dataframe (looks cleaner if you only have a couple of thousand rows of data to process)
# driver.execute_query(
#     ''' 
#         unwind $rows as row
#         merge (n:UseCase{name: row['COL1']})
#             set n.description = row['COL2]
#         return count(*) as rows_processed
#     ''',
#     database_=DB_NAME,
#     routing_=RoutingControl.WRITE,
#     rows = data[['COL1', 'COL1']].drop_duplicates().to_dict('records')
# )

In [12]:
# Policy nodes
for chunk in split_dataframe(csv_policy[['PolicyID','Exposure']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            merge (p:Policy{id: row.PolicyID})
            set p.exposure = row.Exposure
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_processed=5>]
[<Record rows_proces

KeyboardInterrupt: 

In [ ]:
# Car nodes
# PolicyID	ClaimNb	Exposure	Power	CarAge	DriverAge	Brand	Gas	Region	Density
for chunk in split_dataframe(csv_policy[['PolicyID','Power','CarAge','Brand','Gas']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            merge (p:Car{id: row.PolicyID})
            set p.power = row.Power,
                p.age = row.CarAge,
                p.brand = row.Brand,
                p.fuel = row.Gas
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
# Region nodes
# PolicyID	ClaimNb	Exposure	Power	CarAge	DriverAge	Brand	Gas	Region	Density
for chunk in split_dataframe(csv_policy[['Region','Density']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            merge (p:Region{id: row.Region})
            set p.density = row.Density
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
# Claim nodes
# ClaimId	PolicyID	ClaimAmount
for chunk in split_dataframe(csv_claim[['ClaimId','ClaimAmount']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            merge (p:Claim{id: row.ClaimId})
            set p.amount = row.ClaimAmount
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
# (:Policy)-[:CLAIM]->(:Claim) Relationships
# ClaimId	PolicyID	ClaimAmount
for chunk in split_dataframe(csv_claim[['ClaimId','PolicyID']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            match (c:Claim{id: row.ClaimId}), (p:Policy{id: row.PolicyID})
            merge (c)<-[:CLAIM]-(p)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
# (:Policy)-[:ASSET]->(:Car) Relationships
# PolicyID	ClaimNb	Exposure	Power	CarAge	DriverAge	Brand	Gas	Region	Density
for chunk in split_dataframe(csv_policy[['PolicyID']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            match (p:Policy{id: row.PolicyID}), (c:Car{id: row.PolicyID})
            merge (p)-[:ASSET]->(c)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
# (:Policy)-[:GEO]->(:Region) Relationships
# PolicyID	ClaimNb	Exposure	Power	CarAge	DriverAge	Brand	Gas	Region	Density
for chunk in split_dataframe(csv_policy[['PolicyID','Region']].drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            match (p:Policy{id: row.PolicyID}), (c:Region{id: row.Region})
            merge (p)-[:GEO]->(c)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
policy_coverage = pd.read_excel('~/import/PolicyCoverage.xlsx', sheet_name='Sheet1')
policy_coverage.head()

In [ ]:
# Create Product and Coverage part of the graph
for chunk in split_dataframe(policy_coverage.drop_duplicates()):
    records, summary, keys = driver.execute_query(
        ''' 
            unwind $rows as row
            merge (p:Product{id: row.Product})
            merge (c:Coverage{id: row.Coverage})
            merge (p)-[:INCLUDES]->(c)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        rows = chunk.to_dict('records')
    )
    print(records)

In [ ]:
df_claim_amounts = driver.execute_query(
        ''' 
            match (c:Claim)<-[:CLAIM]-(p:Policy)
            where c.amount < 20_000
            return c.amount as amount, p.exposure as exposure
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.READ,
        result_transformer_= lambda r: r.to_df()
    )

In [ ]:
df_claim_amounts.hist()

In [ ]:
# Randomly assign Product to Policy (just for demo)
assignments = [
    {'product': 'Car Super', 'fraction': 0.10*413169},
    {'product': 'Comprehensive' , 'fraction': 0.45*413169},
    {'product': 'Partial-Comprehensive', 'fraction': 0.40*413169},
    {'product': 'Liability', 'fraction': 0.05*413169}
    ]
for assignment in assignments :
    driver.execute_query(
        ''' 
            match (p:Product{id:$product})
            with p
            match (pol:Policy)
            where not exists { (pol)-[:INSTANCE_OF]->(p) }
            with p,pol order by rand() asc limit toInteger($fraction)
            merge (pol)-[:INSTANCE_OF]->(p)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        product = assignment['product'],
        fraction = assignment['fraction']
    )

In [ ]:
# Assign remaining to "Partial-Comprehensive"
driver.execute_query(
        ''' 
            match (p:Product{id:$product})
            with p
            match (pol:Policy)
            where not exists { (pol)-[:INSTANCE_OF]->(p) }
            with p, pol
            merge (pol)-[:INSTANCE_OF]->(p)
            return count(*) as rows_processed
        ''',
        database_=DB_NAME,
        routing_=RoutingControl.WRITE,
        product = 'Partial-Comprehensive'
    )

In [ ]:
# Check assignments
driver.execute_query(
    ''' 
        match (:Policy)-[:INSTANCE_OF]->(p:Product)
        return p.id as product, count(*) as policy_count
        order by policy_count desc
    ''',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
).head()

In [ ]:
# Randomly assign Claims to Coverage
# (:Claim)-[:COVERED_BY]->(:Coverage)
with driver.session(database=DB_NAME) as session:
    session.run(
        ''' 
        match (pol:Policy)-[:CLAIM]->(c:Claim)
        call {
            with pol, c
            with pol, c, apoc.coll.randomItem([ (pol)-[:INSTANCE_OF]->(:Product)-[:INCLUDES]->(cov:Coverage) | cov ]) as cov
            merge (c)-[:COVERED_BY]->(cov)
        } in transactions of 20_000 rows
        ''',
        parameters={'product':'Partial-Comprehensive'}
    ).consume()
    session.close()

# Model so far
![image](./images/insurance-model-start.png)

In [ ]:
# Check what each Coverage costs in paid out claims
driver.execute_query(
    ''' 
        match (c:Claim)<-[:CLAIM]-(:Policy)-[:INSTANCE_OF]->(:Product)-[:INCLUDES]->(cov:Coverage)
        return cov.id as coverage, sum(c.amount)/1.0e6 as claim_amount_mdkr
        order by claim_amount_mdkr desc
    ''',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
).head(20)

In [ ]:
# Refactor, add CarModel nodes
with driver.session(database=DB_NAME) as session:
    session.run(
        ''' 
            match (c:Car)
            call {
                with c
                merge (m:CarModel{id: c.brand + " " + c.fuel})
                merge (c)-[:INSTANCE_OF]->(m)
            } in transactions of 50_000 rows
            return count(*) as rows_processed
        '''
    ).consume()
    session.close()